In [1]:
#%pip install lyricsgenius

In [2]:
import lyricsgenius as lg
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
FOLDER = "data/"

## Data wrangling:
The final data frame **top10\_df** contains top 10 songs per year, sorted by **'Weeks on Chart'**, i.e. the number of weeks they've been on the Billboard chart.

In [4]:
filename = 'billboard.csv'
df = pd.read_csv(FOLDER+filename)
df['Year'] = pd.to_datetime(df['WeekID']).dt.year
df = df.rename(columns={"Week Position":"WeekPosition"})
df = df.drop_duplicates(['SongID'])
df.head()

,url,WeekID,WeekPosition,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,Year
0,http://www.billboard.com/charts/hot-100/1963-0...,6/1/1963,11,Still,Bill Anderson,StillBill Anderson,1.0,17.0,11.0,8.0,1963
1,http://www.billboard.com/charts/hot-100/1967-0...,1/7/1967,11,Coming Home Soldier,Bobby Vinton,Coming Home SoldierBobby Vinton,1.0,17.0,11.0,8.0,1967
2,http://www.billboard.com/charts/hot-100/1971-0...,7/3/1971,11,She's Not Just Another Woman,The 8th Day,She's Not Just Another WomanThe 8th Day,1.0,17.0,11.0,8.0,1971
3,http://www.billboard.com/charts/hot-100/1975-1...,11/29/1975,11,Saturday Night,Bay City Rollers,Saturday NightBay City Rollers,1.0,17.0,11.0,8.0,1975
4,http://www.billboard.com/charts/hot-100/1979-0...,9/29/1979,11,Pop Muzik,M,Pop MuzikM,1.0,17.0,11.0,8.0,1979


In [5]:
df = df.sort_values(by='Weeks on Chart', ascending=False)
top10_df = df.groupby('Year').head(10)
top10_df = top10_df[['Song', 'Performer', 'Year']]
top10_df['Decade'] = (top10_df['Year']//10)*10
top10_df.head()

,Song,Performer,Year,Decade
167831,Eastside,"benny blanco, Halsey & Khalid",2019,2010
252653,Wait For You,Elliott Yamin,2007,2000
271215,Wildflower,Skylark,1973,1970
177187,Even Though I'm Leaving,Luke Combs,2019,2010
315541,Do Re Mi,blackbear,2017,2010


In [6]:
print("Total number of songs: ",len(top10_df))

Total number of songs:  630


## Lyrics scraping:
We get the lyrics for the top_10 songs from Genius API.

We notice that some songs list 2 names of performers, some other list 2 titles (title1/title2), which leads to various errors in the scraping of the lyrics from Genius. So we make some updates:

In [7]:
def update_performer(performer):
    try:
        if performer.find("duet with") > 0:
            performer = performer.split("duet with", 1)[0] #take the performer_1 in "performer_1 duet with performer_2"
        elif performer.find("with") > 0:
            performer = performer.split("with", 1)[0] #take the performer_1 in "performer_1 with performer_2"
        elif performer.find(" and") > 0:
            performer = performer.split(" and", 1)[0]
        elif performer.find("&") > 0:
            performer = performer.split("&", 1)[0]
        elif performer.find("featuring") > 0:
            performer = performer.split("featuring", 1)[0]
        elif performer.find("feat") > 0:
            performer = performer.split("feat", 1)[0]
    except:
        print("Error:", performer)
        
    return performer

In [8]:
top10_df['Performer'] = top10_df['Performer'].str.lower()

top10_df['Performer'] = top10_df.apply(lambda x: update_performer(x['Performer']), axis=1)

In [9]:
def update_song(title):
    try:
        if title.find("/") > 0:
            title = title.split("/", 1)[0] #take the title_1 in "title_1/title_2"
        elif title.find("(from") > 0:
            title = title.split("(from", 1)[0]
    except:
        print("Error:", title)
        
    return title

In [10]:
top10_df['Song'] = top10_df['Song'].str.lower()

top10_df['Song'] = top10_df.apply(lambda x: update_song(x['Song']), axis=1)

We save the lyrics of each song in a column **Lyrics**.

In [11]:
token = 'S7klyjw4NsqkK8U1JXa5SvAZY4ykRU-IEr2lGR8taxKBrtdaZmndGWQCzzbqtdy4'

genius = lg.Genius(token, 
                   skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], 
                   remove_section_headers=True) #ignore metadata

In [12]:
"""
@params: a song title and the performer's name
@return: the lyrics, or "Error" in case of exception
"""
def get_lyrics(song, performer):
    try:
        return genius.search_song(song, performer).lyrics
    except: 
        return "Error"

In [13]:
top10_df['Lyrics'] = top10_df.apply(lambda x: get_lyrics(x['Song'],x['Performer']), axis=1)

Searching for "eastside" by benny blanco, halsey ...
Done.
Searching for "wait for you" by elliott yamin...
Done.
Searching for "wildflower" by skylark...
Done.
Searching for "even though i'm leaving" by luke combs...
Done.
Searching for "do re mi" by blackbear...
Done.
Searching for "you make it easy" by jason aldean...
Done.
Searching for "smack that" by akon ...
Done.
Searching for "alley-oop" by hollywood argyles...
Done.
Searching for "quiet village" by the exotic sounds of martin denny...
Done.
Searching for "the ballad of the green berets" by ssgt barry sadler...
Done.
Searching for "run this town" by jay-z, rihanna ...
Done.
Searching for "poor side of town" by johnny rivers...
Done.
Searching for "blue moon" by the marcels...
Done.
Searching for "burning bridges" by jack scott...
Done.
Searching for "take a bow" by rihanna...
Done.
Searching for "classical gas" by mason williams...
Specified song does not contain lyrics. Rejecting.
Searching for "suspicion" by terry stafford..

In [88]:
top10_df.query('Lyrics=="Error"')

,Song,Performer,Year,Decade,Lyrics
36994,classical gas,mason williams,1968,1960,Error
36771,out of limits,the marketts,1964,1960,Error
36553,panda,desiigner,2016,2010,Error
36561,calcutta,lawrence welk,1961,1960,Error
20966,stir fry,migos,2018,2010,Error
20802,scorpio,dennis coffey,1971,1970,Error
10972,"roc ya body ""mic check 1,2""",m.v.p. (most valuable playas),2003,2000,Error
10985,cups (pitch perfect's when i'm gone),anna kendrick,2013,2010,Error
10601,survival of the fittest,mobb deep,1995,1990,Error
13985,cannonball,duane eddy his twangy guitar,1958,1950,Error


In [15]:
print("Total number of errors: ", len(top10_df.query('Lyrics=="Error"')))

Total number of errors:  21


The number of erronous lyrics is low, we can simply drop them.

We save the final dataframe to a csv file.

In [90]:
filename = "lyrics_per_song.csv"
out_file = open(FOLDER+filename, "w")

top10_df.to_csv(out_file, index=False)

We write the lyrics of the songs from the same decade in a separate file.

# ---------------------------------------------------------------------------------------------------
<br> Fixing wrong lyrics manually:

## 1950

In [16]:
top10_df[top10_df['Lyrics'].str.contains("What parallel courses did Bloom and Stephen follow returning?")]

,Song,Performer,Year,Decade,Lyrics
36911,quiet village,the exotic sounds of martin denny,1959,1950,What parallel courses did Bloom and Stephen fo...


In [17]:
pos = 36911
song = "quiet village"
performer = "martin denny"
top10_df.loc[pos:pos,['Song','Performer', 'Lyrics']] = [song, performer, genius.search_song(song, performer).lyrics]
top10_df[top10_df['Song']==song]

Searching for "quiet village" by martin denny...
Done.


,Song,Performer,Year,Decade,Lyrics
36911,quiet village,martin denny,1959,1950,Alone in my quiet village I pray\nYou will be ...


In [18]:
top10_df[top10_df['Lyrics'].str.contains('The summer evening had begun to fold the world')]

,Song,Performer,Year,Decade,Lyrics
10882,blue ribbon baby,tommy sands,1958,1950,The summer evening had begun to fold the world...


In [19]:
lyrics = "Ooh wee, a blue ribbon baby Ooh wee, you take first prize Ooh wee, a blue ribbon baby You re my baby with The baby blue eyes There s gonna be lots of good Rocking at the dance tonight So tie a big blue ribbon In your hair Not letting my baby Get out of my sight Cause all the guys Are gonna be there Ooh wee, a blue ribbon baby Ooh wee, you take first prize Ooh wee, a blue ribbon baby You  re my baby with The baby blue eyes There  s gonna be Jimmy and Jerry and Jackie and Jill I  m just gonna mention a few And Becky and Buddy And Susie and Phil but all The guys  ll be looking at you Ooh wee, a blue ribbon baby Ooh wee, you take first prize Ooh wee, a blue ribbon baby You  re my baby with The baby blue eyes I  m gonna keep my little baby With the big blue eyes and The big blue ribbon in her hair Cause all along, I knew that she Was gonna win a first prize Don  t you think we  re gonna Make a happy pair Ooh wee, a blue ribbon baby Ooh wee, you take first prize Ooh wee, a blue ribbon baby You  re my baby with The baby blue eyes Oh, my baby with The baby blue eyes Oh, my baby with The baby blue eyes"

In [20]:
pos = 10882
song = "blue ribbon baby"
performer = "tommy sands"
top10_df.loc[pos:pos,['Song','Performer', 'Lyrics']] = [song, performer, lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10882,blue ribbon baby,tommy sands,1958,1950,"Ooh wee, a blue ribbon baby Ooh wee, you take ..."


In [21]:
lyrics = "I work the late shift  At my daddy  s shop  I   ve saved some money  But I  m losing some heart  And every evening  When that whistle blows  I hit the road  I want to just keep going  I met this girl  She lives a ways cross town  She wants somebody  She can settle down  Man I been living in  This town too long  I  d pack my bags up,  I  d be out that door, but  This small town girl  s got me crazy  I  m falling for a small town girl  She  s all that matters  In this whole wide world  Cause I  m falling for  This small town girl  She wants the kind of  Things her mama seen  A little home to  Raise a family  She wears my ring now  And I don  t know why  I  ll be here  Until the day I die,   cause  This small town girl  s got me crazy  I  m falling for a small town girl  She  s all that matters  In this whole wide world…"

In [22]:
pos = 10833
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10833,small town girl,john cafferty,1985,1980,I work the late shift At my daddy s shop I ...


In [23]:
lyrics = "Pussy cat, pussy cat  Pussy cat, pussy catWell, I was born  In a little old house  In a town called Barrel Bay  I was eight years old  When I fell in love and  And it seems like yesterdayDown the old mill road  In a pretty house lived  Some folks not far from me  And they had a child  Only one year old  And a lovely babe to see      They called her Pussy Cat  Puss, Puss, Pussy Cat  For her hair was soft and fine  Pussycat, Puss, Puss, Pussy Cat  For her two big eyes would shine  She had a smile so sweet  And a great deal more than that  And she  d coo and purr  When I smiled at her with  A milky skin like silky fur  My Puss, Puss, Puss, Puss  Pussy CatWell, the years went by  And I grew quite tall  For I shot up like a vine  And I used to watch old Pussy Cat  Till she reached the age of nine  Then I said goodbye  For I left one year,  For a town in Iowa  And I didn  t see anyone I knew    Til I came back one fine day  Well, there she stood  Like a pretty rose and  My goodness, how she  d grown  But the sparkling eyes  And the turned-up nose  Looked the same though  Years had flownThen I knew at once  She belonged to me  And I knew she felt the same  So our lives were joined  And our first new-born  Was a girl who bore her name  My Puss, Puss, Puss, Puss  Pussy Cat, Pussy Cat."

In [24]:
pos = 13948
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
13948,pussy cat,the ames brothers,1958,1950,"Pussy cat, pussy cat Pussy cat, pussy catWell..."


In [25]:
lyrics = "By, by, by the light  Of the silvery moon  I want to croon  With my baby, I  ll croon  Love  s old sweet tune    Honeymoon  Keep on shining in June  Love  s silvery beams  Will bring love  s dreams  We  ll be cuddling soon  By the silvery moon    By, by, by the light  Of the silvery moon  I want to croon  With my honey, I  ll croon  Love  s old sweet tune    Honeymoon  Keep on shining in June  Love  s silvery beams  Will bring love  s dreams  We  ll be cuddling soon  By the silvery moon "

In [26]:
pos = 11145
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
11145,by the light of the silvery moon,jimmy bowen,1958,1950,"By, by, by the light Of the silvery moon I w..."


In [27]:
lyrics = "I m discontented with homes that are rented so I have invented my own Darling this place is a lover s oasis Where lights where we chase is unknown Far from the cry of the city where flowers pretty caress the stream Cozy to hide in, To live side by side in, don t let it abide in my dream Picture you upon my knee, just tea for two and two for tea Just me for you and you for me alone Nobody near us to see us or hear us No friends or relations or weekend vacations We won t have it known, dear, that we own a telephone, dear They will brake and you ll awake and start to bake a sugar cake For me to take for all the boys to see We will raise a family, a boy for you and a girl for me Can t you see how happy we will be "

In [28]:
pos = 20766
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20766,tea for two cha cha,the tommy dorsey orchestra,1958,1950,I m discontented with homes that are rented so...


In [135]:
dec = 1950
filename = "%s_lyrics.txt" % dec
file = open(FOLDER+filename, 'w')
_df = top10_df.query("Decade == @dec")
for l in _df.Lyrics:
    file.write(l+'\n')

## 1960

In [31]:
top10_df[top10_df['Lyrics'].str.contains('SPOKEN:')]

,Song,Performer,Year,Decade,Lyrics
38524,cry baby,garnet mimms,1963,1960,CRY BABY\nGarnet Mimms & the Enchanters\nCry c...


In [32]:
lyrics = "Cry cry baby, cry baby, cry baby, Welcome back home. Now he told you, that he loved you much more than I But he left you and you don t you just don t know why And when you don t know what to do You come running and Cry cry baby, cry baby, cry baby, Welcome back home. Well I can always lose,  Don t you know nobody can love you the way that I do Take the pain and the heartache too Ah honey you know that I ll be around  When you need to Cry cry baby, cry baby, cry baby, Welcome back home. Welcome back home I ve spent so many nights  Just waiting for you to come walking through that door And even tho you ve made a fool of me so many times before  I know that all it takes is just the sight of your face To make me realize that I will always love you darlin And I can see that you got some more tears to shed I can see it baby, cause your eyes, your eyes are getting red So c mon, c mon, c mon, c mon c mon Cry cry baby, cry baby, cry baby, Cry cry baby, cry baby, cry baby,"

In [33]:
pos = 38524
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
38524,cry baby,garnet mimms,1963,1960,"Cry cry baby, cry baby, cry baby, Welcome back..."


In [34]:
lyrics = "I ve got a song, I ain t got no melody I ma gonna sing it to my friends I ve got a song, I ain t got no melody I ma gonna sing it to my friends  Will it go round in circles? Will it fly high like a bird up in the sky? Will it go round in circles? Will it fly high like a bird up in the sky?  I ve got a story, I ain t got no moral Let the bad guy win every once in a while I ve got a story, I ain t got no moral Let the bad guy win every once in a while  Will it go round in circles? Will it fly high like a bird up in the sky? Will it go round in circles? Will it fly high like a bird up in the sky?  I ve got a dance, I ain t got no steps, no I m gonna let the music move me around I ve got a dance, I ain t got no steps I ma gonna let the music move me around  Will it go round in circles? Will it fly high like a bird up in the sky? Will it go round in circles? Will it fly high like a bird up in the sky?  Well, well, now oh Will it go round in circles? Will it fly high like a bird up in the sky? Will it go round in circles? Will it fly high like a bird up in the sky?  I ve got a song, I ain t got no melody I ma gonna sing it to my friends I ve got a song, I ain t got no melody I ma gonna sing it to my friends  Will it go round in circles? Will it fly high like a bird up in the sky? Will it go round in circles? Will it fly high like a bird up in the sky?  Will it go round in circles? Will it fly high like a bird up in the sky? Will it go round in circles? Will it fly high like a bird up in the sky?"

In [35]:
pos = 38603
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
38603,because they're young,duane eddy,1960,1960,"I ve got a song, I ain t got no melody I ma go..."


In [36]:
lyrics = "Badlees, The The Unfortunate Result Of Spare Time The Unfunny Just be clever for clever s sake, starin  at the ceiling into shadow-fakes The circle-smokes in the ladies room only got you as far as the roles assumed A glimpse of the perfect crime is just a reverie away Whadaya say we hang there a while for expediency s sake? And fantasize the great demise of the unfunny Cut and dry, we ll bid good-bye to the unfunny Their personality inspires like a five pound bag of fertilizer Like living in a house with fifteen cats, you re unaware of the stench when a line goes flat A glimpse of the perfect crime is just a reverie away Whadaya say we hang there a while for expediency s sake? And fantasize the great demise of the unfunny Set  em up, we ll drink a cup to the unfunny "

In [37]:
pos = 20570
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20570,pipeline,chantay's,1963,1960,"Badlees, The The Unfortunate Result Of Spare T..."


In [38]:
top10_df[top10_df['Lyrics'].str.contains('In the 1960’s, politics and music had a close relationship')]

,Song,Performer,Year,Decade,Lyrics
36977,the ballad of the green berets,ssgt barry sadler,1966,1960,"In the 1960’s, politics and music had a close ..."


In [39]:
lyrics = "Fighting soldiers from the sky Fearless men who jump and die Men who mean just what they say The brave men of the Green Beret Silver wings upon their chest These are men, America s best One hundred men will test today But only three win the Green Beret Trained to live off nature s land Trained in combat, hand-to-hand Men who fight by night and day Courage take from the Green Berets Silver wings upon their chest These are men, America s best One hundred men will test today But only three win the Green Beret Back at home, a young wife waits Her Green Beret has met his fate He has died for those oppressed Leaving her his last request Put silver wings on my son s chest Make him one of America s best He ll be a man they ll test one day Have him win the Green Beret "

In [40]:
pos = 36977
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
36977,the ballad of the green berets,ssgt barry sadler,1966,1960,Fighting soldiers from the sky Fearless men wh...


In [41]:
top10_df[top10_df['Lyrics'].str.contains('MR. JUSTICE PECKHAM')]

,Song,Performer,Year,Decade,Lyrics
20532,"the ""in"" crowd",ramsey lewis trio,1965,1960,"20 U.S. 123 (1908)MR. JUSTICE PECKHAM, after m..."


In [42]:
lyrics = "I m in with the in crowd  I go where the in crowd goes  I m in with the in crowd  And I know what the in crowd knows   Any time of the year, don t you hear? Dressin  fine, makin  time We breeze up and down the street We get respect from the people we meet They make way day or night They know the in crowd is out of sight  I m in with the in crowd  I know every latest dance  When you re in with the in crowd  It s easy to find romance  At a spot where the beat s really hot Oh, if it s square, we ain t there We make every minute count Our share is always the biggest amount Other guys imitate us But the original s still the greatest  We got our own way of walkin  We got our own way of talkin , yeah  Anytime of the year, don t you hear? Spendin  cash, talkin  trash Girl, I ll show you a real good time Come on with me and leave your troubles behind I don t care where you ve been You ain t been nowhere till you ve been in  With the in crowd, yeah Oh, with the in crowd (yeah, yeah, yeah) We got our own way of walkin , yeah (yeah, yeah, yeah) We got our own way of talkin  (yeah, yeah, yeah) In the in crowd"

In [43]:
pos = 20532
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20532,"the ""in"" crowd",ramsey lewis trio,1965,1960,I m in with the in crowd I go where the in cr...


In [44]:
dec = 1960
filename = "%s_lyrics.txt" % dec
file = open(FOLDER+filename, 'w')
_df = top10_df.query("Decade == @dec")
for l in _df.Lyrics:
    file.write(l+'\n')

## 1970

In [45]:
lyrics = "Ain t no sunshine when she s gone It s not warm when she s away Ain t no sunshine when she s gone And she s always gone too long Anytime she s goes away Wonder this time where she s gone Wonder if she s gone to stay Ain t no sunshine when she s gone And this house just ain t no home Anytime she goes away And I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know Hey I oughta leave young thing alone But ain t no sunshine when she s gone, woh woh Ain t no sunshine when she s gone Only darkness every day Ain t no sunshine when she s gone And this house just ain t no home Anytime she goes away Anytime she goes away Anytime she goes away Anytime she goes away "

In [46]:
pos = 36801
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
36801,ain't no sunshine,bill,1971,1970,Ain t no sunshine when she s gone It s not war...


In [47]:
lyrics = "Ain t no sunshine when she s gone It s not warm when she s away Ain t no sunshine when she s gone And she s always gone too long Anytime she s goes away Wonder this time where she s gone Wonder if she s gone to stay Ain t no sunshine when she s gone And this house just ain t no home Anytime she goes away And I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know, I know, I know, I know I know, I know Hey I oughta leave young thing alone But ain t no sunshine when she s gone, woh woh Ain t no sunshine when she s gone Only darkness every day Ain t no sunshine when she s gone And this house just ain t no home Anytime she goes away Anytime she goes away Anytime she goes away Anytime she goes away "

In [48]:
pos = 20571
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20571,outa-space,billy preston,1972,1970,Ain t no sunshine when she s gone It s not war...


In [49]:
lyrics = "Pick up the pieces, uh, huh Pick up the pieces, alright Pick up the pieces, uh, huh Pick up the pieces, whoo Whoo Pick up the pieces Pick up the pieces Pick up the pieces, whoo Pick up the pieces, oow"

In [50]:
pos = 20492
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20492,pick up the pieces,awb,1975,1970,"Pick up the pieces, uh, huh Pick up the pieces..."


In [51]:
lyrics = "We all came out to Montreux On the Lake Geneva shoreline To make records with a mobile We didn t have much time Frank Zappa and the Mothers Were at the best place around But some stupid with a flare gun Burned the place to the ground Smoke on the water A fire in the sky Smoke on the water They burned down the gambling house It died with an awful sound Uh, Funky Claude was running in and out Pulling kids on the ground When it all was over We had to find another place But Swiss time was running out It seemed that we would lose the race Smoke on the water A fire in the sky Smoke on the water We ended up at the Grand Hotel It was empty, cold, and bare But with the Rolling truck Stones thing just outside Making our music there With a few red lights, a few old beds We make a place to sweat No matter what we get out of this Ha, I know, I know we ll never forget Smoke on the water A fire in the sky Smoke on the water"

In [52]:
pos = 20653
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20653,smoke on the water,deep purple,1973,1970,We all came out to Montreux On the Lake Geneva...


In [53]:
lyrics = "Like a fool i went and stayed too long Now i m wondering if your love s still strong Oo baby, here i am, signed, sealed delivered, i m yours Then that time i went and said goodbye Now i m back and not ashamed to cry Oo baby, here i am, signed, sealed delivered, i m yours Here i am baby Oh, you ve got the future in your hand (Signed, sealed delivered, i m yours) here i am baby, Oh, you ve got the future in your hand (Signed, sealed, delivered, i m yours) I ve done alot of foolish things That i really didn t mean Hey, hey, yea, yea, didn t i, oh baby Seen a lot of things in this old world When i touched them they did nothing, girl Oo baby, here i am, signed, sealed delivered, i m yours, oh i m yours Oo-wee babe you set my soul on fire That s why i know you are my only desire Oo baby, here i am, signed, sealed delivered, i m yours Here i am baby Oh, you ve got the future in your hand (Signed, sealed delivered, i m yours) here i am baby, Oh, you ve got the future in your hand (Signed, sealed, delivered, i m yours) I ve done alot of foolish things That i really didn t mean I could be a broken man but here i am With your future, got your future babe (here i am baby) Here i am baby (signed, sealed delivered, i m yours) Here i am baby, (here i am baby) Here i am baby (signed, sealed delivered, i m yours) Here i am baby, (here i am baby) Here i am baby (signed, sealed delivered, i m yours)"

In [54]:
pos = 14209
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
14209,"signed, sealed, delivered (i'm yours)",peter frampton,1977,1970,Like a fool i went and stayed too long Now i m...


In [55]:
top10_df[top10_df['Lyrics'].str.contains('"Aberfan" -- by Dulahan -- About the 1966')]

,Song,Performer,Year,Decade,Lyrics
20469,space race,billy preston,1973,1970,All these songs are about (or heavily referenc...


In [56]:
lyrics = "How men be so greedy When there s so much left? All things are God-given And they all have been blessed Why can t we be humble Like the good Lord say? He promised to exalt us But love is the way That s the way God planned it That s the way God wants it to be Oh, that s the way God planned it That s the way God wants it to be Let not your heart be troubled Let mourning, sobbing cease Learn to love your brothers and And live in perfect peace Why can t we be humble Like my Lord say? He promised to exalt us But love is the way That s the way God planned it That s the way God wants it to be Yeah, that s the way God planned it That s the way God wants it to be Oh, why can t we be humble Like my Lord said, y all? God s gonna exalt us all But love is the way Yeah, that s the way God planned it That s the way God wants it to be Ya better believe me That s the way God planned it That s the way God wants it to be, ya-hey Why don t you believe me? That s the way God planned it That s the way He wants us to be Love one another Whoa, man That s the way God planned it That s the way God wants it to be So we can have peace now We ve gotta do it the way God planned it We gotta do it way He planned it to be Before we can have peace, y all We ve gotta do it the way God planned it We gotta do it the way God wants it to be That s the way God planned it"

In [57]:
pos = 20469
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20469,space race,billy preston,1973,1970,How men be so greedy When there s so much left...


In [58]:
dec = 1970
filename = "%s_lyrics.txt" % dec
file = open(FOLDER+filename, 'w')
_df = top10_df.query("Decade == @dec")
for l in _df.Lyrics:
    file.write(l+'\n')

## 1980

In [59]:
lyrics = "Oh, baby now let s get down tonight (ba-ba-ba, bum) Ooh baby, I m hot just like an oven (heal me my darling, heal me my darling) I need some lovin  And baby, I can t hold it much longer (heal me my darling, heal me my darling) It s getting stronger and stronger When I get that feeling I want sexual healing Sexual healing, oh baby Makes me feel so fine Helps to relieve my mind Sexual healing baby, is good for me Sexual healing is something that s good for me Whenever blue teardrops are fallin  And my emotional stability is leaving me There is something I can do I can get on the telephone and call you up, baby And honey I know you ll be there to relieve me The love you give to me will free me If you don t know the thing you re dealing Oh I can tell you, darling, that it s sexual healing Let s make love tonight (Wake up, wake up, wake up, wake up)  Cause you do it right Baby, I got sick this mornin  (heal me my darling, heal me my darling) A sea was stormin  inside of me Baby, I think I m capsizin (heal me my darling, heal me my darling) The waves are risin  and risin  And when I get that feeling I want sexual healing Sexual healing is good for me Makes me feel so fine, it s such a rush Helps to relieve the mind, and it s good for us Sexual healing, baby, it s good for me Sexual healing is something that s good for me Well, it s good for me and it s so good to me my baby, oh Come take control, just grab a hold Of my body and mind, soon we ll be making it, honey I ll be feeling fine You re my medicine, open up and let me in Darling, you re so great, I can t wait for you to operate (Heal me my darling) (Heal me my darling) I can t wait for you to operate When I get this feeling (heal me my darling) I need sexual healing Oh, when I get this feeling (heal me my darling) I need sexual healing  I gotta have sexual healing, darling (heal me my darling)  Cause I m all alone  I need Sexual healing, darling (heal me my darling)  Til you come back home (heal me my darling) Please don t procrastinate It s not good to masturbate"

In [60]:
pos = 20659
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20659,sexual healing,marvin gaye,1982,1980,"Oh, baby now let s get down tonight (ba-ba-ba,..."


In [61]:
lyrics = "Proud Carry your head so high Feeling kinda of mighty Maybe take a little flight  Proud Who do you think you are Gotta getta grip Or you run out of light  Proud Carry your head so high Feeling kinda of mighty Maybe take a little flight  Proud Who do you think you are Gotta getta grip Or you run out of light  If you feel the urge to groove you better listen When you do experience the strangest feeling You are born to think it s a matter of fact Let your conscience be your guide  Who are you impressing with your lack of virtue Don t you think you ve had to the time to learn the true things You re a bigger fool if you really believe You ll find a diamond in the sky  So proud Carry your head so high Feeling kinda of mighty Maybe take a little flight  Proud Who do you think you are Gotta getta grip Or you run out of light  Open up your eyes and use your inner vision Everybody needs some kind of confirmation So you begin to think a pat on the back Is all you need to get you back  You feed your ego Glorified you, critizied without a reason And you groan and weep And will never cease to have a cautious state of mind  So proud Carry your head so high Feeling kinda of mighty Maybe take a little flight  Proud Who do you think you are Gotta getta grip Or you run out of light  Proud Carry your head so high Feeling kinda of mighty Maybe take a little flight  Proud Who do you think you are Gotta getta grip Or you run out of light  Proud Proud So so so proud "

In [62]:
pos = 11039
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
11039,proud,the joe chemay band,1981,1980,Proud Carry your head so high Feeling kinda of...


In [63]:
top10_df[top10_df['Lyrics'].str.contains("Rob Markman")]

,Song,Performer,Year,Decade,Lyrics
20586,abracadabra,the steve miller band,1982,1980,"8/1\nRob Markman - ""Easy Money Sniper""\nvowl. ..."


In [64]:
lyrics = "I heat up, I can t cool down  You got me spinnin    Round and  round   Round and  round and  round it goes  Where it stops nobody knows Every time you call my name  I heat up like a burnin  flame  Burnin  flame full of desire  Kiss me baby, let the fire get higher Abra-abra-cadabra  I want to reach out and grab ya  Abra-abra-cadabra  Abracadabra You make me hot, you make me sigh  You make me laugh, you make me cry  Keep me burnin  for your love  With the touch of a velvet glove Abra-abra-cadabra  I want to reach out and grab ya  Abra-abra-cadabra  Abracadabra I feel the magic in your caress  I feel magic when I touch your dress  Silk and satin, leather and lace  Black panties with an angel s face I see magic in your eyes  I hear the magic in your sighs  Just when I think I m gonna get away  I hear those words that you always say Abra-abra-cadabra  I want to reach out and grab ya  Abra-abra-cadabra  Abracadabra Every time you call my name  I heat up like a burnin  flame  Burnin  flame full of desire  Kiss me baby, let the fire get higher I heat up, I can t cool down  My situation goes  round and  round  I heat up, I can t cool down  My situation goes  round and  round  I heat up, I can t cool down  My situation goes  round and  round"

In [65]:
pos = 20586
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20586,abracadabra,the steve miller band,1982,1980,"I heat up, I can t cool down You got me spinn..."


In [66]:
top10_df[top10_df['Lyrics'].str.contains('Chapter 1vandalizing the standards we are held to')]

,Song,Performer,Year,Decade,Lyrics
10962,brite eyes,robbin thompson band,1980,1980,DIRTIERNASTIER$UICIDE Lyric Book\nan examinati...


In [67]:
lyrics = "Just a young child, you ve been runnin wild You saw the bright lights and they amazed you You said you had a good time, do goodtimes make you cry Brite eyes, I think they confused you You ve got to see things for yourself You ain t livin for no body else So go on and take it, you can make it You say they don t understand, they guide you with a heavy hand Brite eyes, can t you see their afraid too They watch you growin  up, it s the one thing they just can t stop Brite eyes their afraid of loosin  you (Chorus) you ve got to see things for yourself You ain t livin for no body else So go on and take it, you can make it Right or wrong, be strong, it ll come along When that road gets hard to find, you ll be lookin for someone down the line But no one will be there to see you through, honey what you gonna do? (Chorus) Brite eyes don t cry (x4) It s gonna be alright I m gonna take you on a midnight flight You re gonna grow up overnight It s gonna be alright It s gonna be alright"

In [68]:
pos = 10962
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10962,brite eyes,robbin thompson band,1980,1980,"Just a young child, you ve been runnin wild Yo..."


In [69]:
top10_df[top10_df['Lyrics'].str.contains('In the name of Annah the Allmaziful')]

,Song,Performer,Year,Decade,Lyrics


In [70]:
lyrics = "Mayday Come in, commander Mayday, commander Mission control Gravity zero Pray for my soul Tell my family I love them Don t forget to kiss the children Tell me, why do I have to be the one? Gone beyond my limit, hotter than the sun No, I can t handle this at all Got kinetic fever and I m bouncin  off the walls Mayday, commander I m losing my mind Break up the silence Give me a sign Give me static, give me noises Let me hear some human voices Tell me, why do I have to be the one? Gone beyond my limit, hotter than the sun No, I can t handle this at all Got kinetic fever and I m bouncin  off the walls Boom, boom, boom, boom, boom Boom, boom, boom, boom, boom, boom Boom, boom, boom, boom, boom Bouncin  off the walls Boom, boom, boom, boom, boom Boom, boom, boom, boom, boom, boom Boom, boom, boom, boom, boom Bouncin  off the walls Boom, boom, boom, boom, boom Hey, hey, hey, yeah Tell me, why do I have to be the one? Gone beyond my limit, hotter than the sun No, I can t handle this at all Got kinetic fever and I m bouncin  off the walls Boom, boom, boom, boom, boom Boom, boom, boom, boom, boom, boom Boom, boom, boom, boom, boom Bouncin  off the walls Boom, boom, boom, boom, boom Boom, boom, boom, boom, boom, boom Boom, boom, boom, boom, boom Bouncin  off the walls Boom, boom, boom, boom, boom Boom, boom, boom, boom, boom, boom Boom, boom, boom, boom, boom Bouncin  off the walls Boom, boom, boom, boom, boom Boom, boom, boom, boom, boom, boom Boom, boom, boom, boom, boom Bouncin  off the walls Boom, boom, boom, boom, boom (Hey, hey, hey, yeah) Boom, boom, boom, boom, boom, boom Boom, boom, boom, boom, boom Bouncin  off the walls Boom, boom, boom, boom, boom (Boom, boom, boom) Boom, boom, boom, boom, boom, boom (Boom, boom, boom, boom, boom) Boom, boom, boom, boom, boom Bouncin  off the walls Mayday I m falling, falling, falling, falling, falling Ow Boom, boom, boom, boom Bou-bou-bouncin , bouncin , bounce, bouncin  off Bouncin  off, bounce, bounce, bouncin  off the wa- Bouncin  off the, bouncin  off the walls"

In [71]:
pos = 10722
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10722,bouncing off the wall,matthew wilder,1984,1980,"Mayday Come in, commander Mayday, commander Mi..."


In [72]:
top10_df[top10_df['Lyrics'].str.contains('Well, Michelle Shocked,')]

,Song,Performer,Year,Decade,Lyrics
11036,anchorage,michelle shocked,1989,1980,"Well, Michelle Shocked, I am actually totally ..."


In [73]:
lyrics = "I took time out to write to my old friend I walked across that burning bridge Mailed my letter off to Dallas But her reply came from Anchorage, Alaska  She said:  Hey girl, it s about time you wrote It s been over two years you know, my old friend Take me back to the days of the foreign telegrams And the all-night rock and rollin ... hey Shell We was wild then  Hey Shell, you know it s kind of funny Texas always seemed so big But you know you re in the largest state in the union When you re anchored down in Anchorage  Hey Girl, I think the last time I saw you Was on me and Leroy s wedding day What was the name of that love song they played? I forgot how it goes I don t recall how it goes  Anchorage Anchored down in Anchorage  Leroy got a better job so we moved Kevin lost a tooth now he s started school I got a brand new eight month old baby girl I sound like a housewife Hey Shell, I think I m a housewife  Hey Girl, what s it like to be in New York? New York City - imagine that! Tell me, what s it like to be a skateboard punk rocker?  Leroy says  Send a picture  Leroy says  Hello  Leroy says  Oh, keep on rocking, girl   yeah, keep on rocking   Hey Shell, you know it s kind of funny Texas always seemed so big But you know you re in the largest state in the union  When you re anchored down in Anchorage Oh, Anchorage Anchored down in Anchorage Oh, Anchorage  "

In [74]:
pos = 11036
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
11036,anchorage,michelle shocked,1989,1980,I took time out to write to my old friend I wa...


In [75]:
top10_df[top10_df['Lyrics'].str.contains('I wanna thank Drama')]

,Song,Performer,Year,Decade,Lyrics
11064,papa was a rollin' stone,wolf,1983,1980,"Gangster, gangster\nDatPiff dot com, world pre..."


In [76]:
lyrics = "It was the third of September; that day I ll always remember,  Cause that was the day that my daddy died. I never got a chance to see him; Never heard nothin  but bad things about him. Mama I m depending on you to tell me the truth. Mama just looked at him and said,  Son, Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone.  Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone.  Hey, Mama, is it true what the say, That Papa never worked a day in his life? And Mama, they talk all around town say that Papa had three outside children and another wife And that ain t right. Heard them talkin  about Papa doing some storefront preachin  Talkin  about saving your souls and All the time weak, dealin  in death And stealin  in the name of the Lord Mama just hung her head and said, Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone.  Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone.  Hey, Mama, I Heard Papa call himself a jack of all trades. Tell me, is that what sent Papa to an early grave? Folks say Papa would beg; borrow or steal to pay his bills. Hey, Mama, folks say Papa was never much on thinkin ; Spend most of his time chasin  women and drinkin ! Mama, I m depending on you to tell me the truth. Mama just hung her head and said,  Son, Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone.  Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone.  Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone.  Papa was a rollin  stone. Wherever he laid his head was his home. And when he died, all he left us was alone. "

In [77]:
pos = 11064
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
11064,papa was a rollin' stone,wolf,1983,1980,It was the third of September; that day I ll a...


In [78]:
dec = 1980
filename = "%s_lyrics.txt" % dec
file = open(FOLDER+filename, 'w')
_df = top10_df.query("Decade == @dec")
for l in _df.Lyrics:
    file.write(l+'\n')

## 1990

In [79]:
lyrics = "Nothing really matters till you make it Nothing really matters till in the end The world still turns and life you learn Nothing really matters in the end You sure about this, baby (That s the way of the world)  Cause I been feeling Down for a while now I better do something real soon (That s the way of the world) I want out of this life Maybe even commit suicide Nothing really matters till you make it (All right, all right) Nothing really matters till the end Your love s all gone but keep it strong And nothing really matters in the end I don t think that s me, baby (That s the way of the world) I just feeling down, boy Got no friends in my life, boy What s going on here (That s the way of the world) Things better change for me And they have to change fast Nothing really matters till you show it Nothing really matters till you say I tell you now when you let me down Nothing really matters in the end All right, tell me again, baby I needed you but you could not be found (That s the way of the world) Strolling through problems World dragging us down Trying to solve all these troubles we made (That s the way of the world) Stuck in the middle of life It s a shame I hope what you saying to me Is real true, baby I know what I m gonna do (That s the way of the world) (That s the way of the world) You re the only friend I got, baby With Cathy Dennis It s about time we should get it on (That s the way of the world) (That s the way of the world) At an all-time low, baby Nothing really matters till you make it Nothing really matters till in the end The world still turns and life you learn But nothing really matters in the end Nothing really matters in the end (That s the way of the world) The end, the end (That s the way of the world) The end, the end"

In [80]:
pos = 10943
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10943,that's the way of the world,d-mob,1990,1990,Nothing really matters till you make it Nothin...


In [81]:
lyrics = "Yabba dabba doo! Flintstones, meet the Flintstones They re the modern stone age family From the town of Bedrock They re a page right out of history Let s ride with the family down the street Through the courtesy of Fred s two feet When you re with the Flintstones Have a yabba dabba doo time A dabba doo time We ll have a gay old time The Flintstones! Flintstones, meet the Flintstones They re the modern stone age family From the town of Bedrock They re a page right out of history Someday, maybe Fred will win the fight Then the cat will stay out for the night When you re with the Flintstones Have a yabba dabba doo time A dabba doo time We ll have a gay old time We ll have a gay old time (yeah)"

In [82]:
pos = 11005
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
11005,(meet) the flintstones,the b.c. 52's,1994,1990,"Yabba dabba doo! Flintstones, meet the Flintst..."


In [83]:
top10_df[top10_df['Lyrics'].str.contains('Justice')]

,Song,Performer,Year,Decade,Lyrics
20964,"bad boys (theme from ""cops"")",inner circle,1993,1990,As days go by in a rush\nYou've forgotten why ...
20485,sadeness part 1,enigma,1991,1990,"Second Essay\nGuilt, Bad Conscience, and Relat..."


In [84]:
lyrics = "Bad boys, whatcha want Watcha want, whatcha gonna do? When sheriff John Brown come for you Tell me whatcha wanna do, whatcha gonna do? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? When you were eight and you had bad traits You go to school and learn the golden rule So why are you acting like a bloody fool? If you get hot, you must get cool Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? You chuck it on that one You chuck it on this one You chuck it on your mother And you chuck it on your father You chuck it on your brother And you chuck it on your sister You chuck it on that one And you chuck it on me Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Nobody now give you no break Police now give you no break Not soldier man give you no break Not even you idren now give you no breaks Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Why did you have to act so mean? Don t you know you re human being? Born of a mother with the love of a father Reflections come and reflections go I know sometimes you wanna let go I know sometimes you want to let go Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? You re too bad, you re too rude You re too bad, you re too rude Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you? You chuck it on that one You chuck it on this one You chuck it on your mother And you chuck it on your father You chuck it on your brother And you chuck it on your sister You chuck it on that one And you chuck it on me Bad boys, bad boys whatcha gonna do? Whatcha gonna do when they come for you?"

In [85]:
pos = 20964
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20964,"bad boys (theme from ""cops"")",inner circle,1993,1990,"Bad boys, whatcha want Watcha want, whatcha go..."


In [100]:
lyrics = "Yeah. sendin this one out. to my man Killa B No doubt indeed. without weed. knowhatI msayin? That old real shit. There s a war goin on outside no man is safe from You could run but you can t hide forever From these streets that we done took You walkin witcha head down scared to look You scause ain t no such things as halfway crooks They never around when the beef cooks in my part of town It s similar to Vietnam Now we all grown up and old, and beyond the cop s control They better have the riot gear ready Tryin to bag me and get rocked steady By the mac one-double, I touch you And leave you with not much to go home wit My skin is thick, cause I be up in the mix of action If I m not at home, puffin lye relaxin New York got a nigga depressed So I wear a slug-proof underneath my Guess God bless my soul, before I put my foot down and begin to stroll And to the drama I built, and all unfinished beef You will soon be killed, put us together It s like mixin vodka and milk I m goin out blastin, takin my enemies with me And if not, they scarred, so they will never forget me Lord forgive me the Hennesey got me not knowin how to act I m fallin and I can t turn back Or maybe it s the words from my man Killa Black That I can t say so it s left a untold fact, until my death My goal s to stay alive Survival of the fit only the strong survive   Yo, yo We livin this til the day that we die Survival of the fit only the strong survive (We still livin it) We livin this til the day that we die Survival of the fit only the strong survive (Thug life, we still livin it) We livin this til the day that we die Survival of the fit only the strong survive (We still livin it) We livin this til the day that we die (We livin this til the day that we die) Survival of the fit only the strong survive (Survival of the fit only the strong.) I m trapped, in between two worlds, tryin to get dough y know When the dough get low the jewels go, but never that As long as fiends smoke crack I ll be on the block hustlin countin my stacks No doubt, watchin my back and proceed with caution Five-oh lurkin, no time to get lost in -- the system Niggaz usin fake names to get out quick My brother did it and got bagged with two ounces I-llegal world where squads hit the block hard Ask my man Twin when he got bagged, that fucked me up God But shit happens for a reason You find out who s your true peoples when you re upstate bleedin You can t find a shorty to troop your bid witchu Hit wit a 2 to 4 it s difficult Wild on the streets I try to maintain Tight with my loot, cause hoes like to run game Some niggaz like to trick but I ain t wit that trickin shit I m like a Jew, savin dough so I can big whip Pushin a Lex, now I m set, ready to jet No matter how much loot I get I m stayin in the projects, forever Jakes on the blocks we out-clever If beef, we never seperate and pull together When worse comes to worse and my peoples come first Try to react and get them motherfuckin feelings hurt My crew s all about loot Fuck lookin cute, I m strictly Timb boots and army certified suits Puffin L s, laid back, enjoyin the smell In the Bridge gettin down it ain t hard to tell You better realize   We livin this til the day that we die Survival of the fit only the strong survive (We still livin it) We livin this til the day that we die Survival of the fit only the strong survive (Thug life, we still livin it) We livin this til the day that we die Survival of the fit only the strong survive (We still livin it) We livin this til the day that we die Survival of the fit only the strong survive (Thug life, we still livin it) (The strong survive) Look in the eyes and get wise Look alive, in ninety-five, word up Hypnotic thug life, get that ass paralyzed Knahmsayin?   and all that "

In [101]:
pos = 10601
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10601,survival of the fittest,mobb deep,1995,1990,Yeah. sendin this one out. to my man Killa B N...


In [134]:
dec = 1990
filename = "%s_lyrics.txt" % dec
file = open(FOLDER+filename, 'w')
_df = top10_df.query("Decade == @dec")
for l in _df.Lyrics:
    file.write(l+'\n')

## 2000

In [107]:
lyrics = "  Roc ya body, mic check  ,  cuz it aint a party until my crew run through Shake somebody, show me watcha can do, like oooh oooh Roc ya body, mic check  ,  DJ spin the music back into the groove Pump it up so the crowd wan? move, like oooh oooh      Check, now, I blaze the illist, so much skillis, impossible for u not to feel us the Realist MC that chu ever met in ur life, I m tryin to see the work, put a lil sweat in ur life And I got an idea, and it might sound silly But I wanna roll ur body in a tight brown philly Crack it, lick it up, seal it n tight, get a light Cuz u know we gonna burn it all night And I just might, double the ice on dent So when u double the price, the advice ride in, c mon    Roc ya body, mic check  ,  cuz it aint a party until my crew run through Shake somebody, show me watcha can do, like oooh oooh Roc ya body mic check  ,  DJ spin the music back into the groove Pump it up so the crowd wan? move, like oooh oooh      Like boom, I ll be at the hotel soon We gon? put the lens on zoom, benz on vroom As long as u got the right perfume Aint no body checking out the telly till noon U should get a sack tack, on the backbone Stand back, I m ill enough with tight sad songs Honey stop breathing when I step in the room And aint no body leaving when I set in the boom    Roc ya body, mic check  ,  cuz it aint a party until my crew run through Shake somebody, show me watcha can do, like oooh oooh Roc ya body mic check  ,  DJ spin the music back into the groove Pump it up so the crowd wan? move, like oooh oooh I like it when u work for me, lemme see u do that boom boom I like the way u work that for me And if u wanna party baby we can get together boom boom Cuz u know u make me so horny And baby u a hottie when u get up on the floor and boom boom Never wanted someone so badly We can leave the club and get a telly get a room boom boom leave the club, get a telly, get a room boom boom (clapping) lets go (more clapping) all my ladies, sing along, c mon  we can party until the night is through so baby tell me what chu wanna do got the keys to the hotel room u know I got my eyes on u hands rubbin up and down the my thighs staring at chu with my bedroom eyes the way u lookin boy u so so fine so we can get it on tonite      Yea, now, this rhyme is for the masters Excused to use  pac homes to teach classes yo Try to follow when the turn tables spinnin The sound is even loud in the crowd gettin up in it U see me postin, the scene get frozen, we sound dope when the words r composed and dis boys break down so wit the slow jam I be so damn ill till I m an old man Stop, ya makin it hot when ya move Stag, I reach the top cuz I choose Grooves, cracks like a cat like u s In fact the mics with the rap cant lose    Roc ya body, mic check  ,  cuz it aint a party until my crew run through Shake somebody, show me watcha can do, like oooh oooh Roc ya body, mic check  ,  cuz it aint a party until my crew run through Shake somebody, show me watcha can do, like oooh oooh Roc ya body mic check  ,  DJ spin the music back into the groove Pump it up so the crowd wan? move, like oooh oooh"

In [108]:
pos = 10972
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10972,"roc ya body ""mic check 1,2""",m.v.p. (most valuable playas),2003,2000,"Roc ya body, mic check , cuz it aint a par..."


In [110]:
dec = 2000
filename = "%s_lyrics.txt" % dec
file = open(FOLDER+filename, 'w')
_df = top10_df.query("Decade == @dec")
for l in _df.Lyrics:
    file.write(l+'\n')

## 2010

In [112]:
lyrics = "You are now listening to the hottest Producer With the hottest beats Calling that, Madness This what they all been waitin  for I guess so They been waitin  for this shit for a long time didn t they, hey I guess so Been waitin  for this shit for a long time didn t they, hey Time to share my vibe right now I m feelin  myself Panda, Panda Panda, Panda, Panda, Panda, Panda I got broads in Atlanta Twistin  dope, lean, and the Fanta Credit cards and the scammers Hittin  off licks in the bando Black X6, Phantom White X6 looks like a panda Goin  out like I m Montana Hundred killers, hundred hammers Black X6, Phantom White X6, panda Pockets swole, Danny Sellin  bar, candy Man I m the macho like Randy The choppa go Oscar for Grammy Damn nigga pull up ya panty Hope you killas understand me I got broads in Atlanta Twistin  dope, lean, and the Fanta Credit cards and the scammers Hittin  off licks in the bando Black X6, Phantom White X6 looks like a panda Goin  out like I m Montana Hundred killers, hundred hammers Black X6, Phantom White X6, panda Pockets swole, Danny Sellin  bar, candy Man I m the macho like Randy The choppa go Oscar for Grammy Bitch nigga pull up ya panty Hope you killas understand me Hey, hmm Panda, Panda Panda, Panda, Panda, Panda, Panda, Panda, Panda, Panda I got broads in Atlanta Twistin  dope, lean, and shit, sippin  Fanta Credit cards and the scammers Wake up Versace shit, life Desiigner Whole bunch of lavish shit They be askin   round town who be clappin  shit I be pullin  up stuff in the Phantom ship I got plenty of stuff of Bugatti whip look how I drive this shit Black X6, Phantom White X6, killin  on camera Pop a Perc, I can t stand up Gorilla, they come and kill you with bananas Four fillas, they finna pull up in the Phantom Know niggas, they come and kill you on the camera Big Rollie, it dancin  bigger than a Pandie Go Oscar for Grammy, bitch pull up your panty Fill up I ma flip it, I got bitches pull up and they get it I got niggas that s countin  for digits Say you make you a lot of new money Know some killers pull off and they in the Wraith CDG, they pull off and they kill the Bape Call up Phillip-Phillip, gon  fill the bank Niggas up in the bank, we gon  drill the bank Fuck we gon  kill the bank, get it I got broads, yea I get it I get cards yea I shitted This how I live it Did it all for a ticket Now Flex drop bombs when he spin it And Bobby gon  trend it Jeff The Don doin  business Zana Ray fuckin  up shit and she doin  her bidnezz I be gettin  to the chicken Countin  to the chicken And all of my niggas gon  split it Panda, Panda Panda, Panda, Panda, Panda, Panda I got broads in Atlanta Twistin  dope, lean, and the Fanta Credit cards and the scammers Hittin  off licks in the bando Black X6, Phantom White X6 looks like a panda Goin  out like I m Montana Hundred killers, hundred hammers Black X6, Phantom White X6, panda Pockets swole, Danny Sellin  bar, candy Man I m the macho like Randy The choppa go Oscar for Grammy Bitch nigga pull up ya panty Hope you killas understand me I got broads in Atlanta Twistin  dope, lean, and the Fanta Credit cards and the scammers Hittin  off licks in the bando Black X6, Phantom White X6 looks like a panda Goin  out like I m Montana Hundred killers, hundred hammers Black X6, Phantom White X6, panda Pockets swole, Danny Sellin  bar, candy Man I m the macho like Randy The choppa go Oscar for Grammy Bitch nigga pull up ya panty Hope you killas understand me Panda, Panda Panda, Panda, Panda, Panda, Panda"

In [113]:
pos = 36553
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
36553,panda,desiigner,2016,2010,You are now listening to the hottest Producer ...


In [114]:
lyrics = "Woo, woo, woo, woo Woo, woo, woo, woo Dance with my dogs in the nighttime (wroof) Trap nigga with the chickens like Popeye s (Popeye s) Money changin  colors like tie-dye (tie-dye) I m just tryna get it, I ain t tryna die (whoa) She got a big ol  onion booty, make the world cry (cry) In the kitchen, wrist twistin  like a stir fry (whip it) Hold them bands down (hey) Hold your mans down (hey) Who told you come around? (who?) This that trap sound (trap) Designer clothes (clothes), fashion shows (shows) Trap, house (house), made of gold (gold) Control the bag now (control it) No need to brag now (no need) Ayy, put the mask down (mask) We livin  fast now (fast) Finest hoes (finest), wrist froze (froze) We can go (we can), coast to coast (coast) Watchin  me whip up, still be real and famous Dance with my dogs in the nighttime (yeah, woo, woo, woo, roof) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) (Offset) Take it, whip it, intermission, let the birds fly (brrr) I get money, tunnel vision through my third eye (money) In that skillet, watch me flip it like it s Five Guys (flip it) Look at my pilgrim, check out the ceiling, look at the blue sky (skrrt) Icy Patek, check, yeah, boogers, they sit on my neck (ice) I don t regret shit, yeah, I m petty and I don t got a debt You crawl  fore you walk, you can ball every week For my dogs  hind the wall, we gon  ball when you free (ball) Stop watchin  me, democracy, you wanna copy me (you watchin ) Life s Monopoly, go cop me some land and some property (property) AP, Rolls, pink diamonds, whoa Whip up the soda, diamonds off the Royce (hey) Keep watchin  me whip up, still be real and famous Dance with my dogs in the night time, yeah (wroof) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (wroof) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, curry chicken, call it stir fry Takeoff, I m the bird keeper, let the birds fly Why s tryna take my prize? You a dead guy Of course, I gotta keep watch out through my bird eye No casket, drop dead fresh and I got dead guys Don t discriminate, ball players come in all sizes Finger roll, post move or the pick and roll They mad the way we win, they think we used a cheat code Why you keep lookin  at me? I feel like niggas got static It must be the Patek, pockets blue cheese, I m in the kitchen In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) Keep watchin  me whip up, still be real and famous Dance with my dogs in the night time yeah (wroof) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (whip it) In the kitchen, wrist twistin  like a stir fry (wroof)"

In [115]:
pos = 20966
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
20966,stir fry,migos,2018,2010,"Woo, woo, woo, woo Woo, woo, woo, woo Dance wi..."


In [116]:
lyrics = "I got my ticket for the long way  round Two bottle o  whiskey for the way And I sure would like some sweet company And I m leavin  tomorrow, what do you say? When I m gone When I m gone You re gonna miss me when I m gone You re gonna miss me by my hair You re gonna miss me everywhere, oh You re gonna miss me when I m gone When I m gone When I m gone You re gonna miss me when I m gone You re gonna miss me by my walk You re gonna miss me by my talk, oh You re gonna miss me when I m gone I got my ticket for the long way  round The one with the prettiest of views It s got mountains, it s got rivers It s got sights give you shivers But it sure would be prettier with you When I m gone When I m gone You re gonna miss me when I m gone You re gonna miss me by my walk You re gonna miss me by my talk, oh You re gonna miss me when I m gone When I m gone When I m gone You re gonna miss me when I m gone You re gonna miss me by my hair You re gonna miss me everywhere Oh, you re sure gonna miss me when I m gone When I m gone When I m gone You re gonna miss me when I m gone You re gonna miss me by my walk You re gonna miss me by my talk, oh You re gonna miss me when I m gone "

In [117]:
pos = 10985
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10985,cups (pitch perfect's when i'm gone),anna kendrick,2013,2010,I got my ticket for the long way round Two bo...


In [120]:
top10_df[top10_df['Lyrics'].str.contains('5/1')]

,Song,Performer,Year,Decade,Lyrics
10639,soltera,"lunay, daddy yankee",2019,2010,"5/1\nAsian Da Brat - ""Tweakin""\nBanda Los Seba..."


In [121]:
lyrics = "Lu-lu-lunay Ella lo que quiere e  joder, vacilar Solita pa  romper la disco Ella lo que quiere e  joder, vacilar (Chi-chi-chium) Dale hasta abajo, pa  hasta abajo sin parar Y estar soltera está de moda Por eso ella no se enamora Y estar soltera está de moda Por eso no va a cambiar Estar soltera está de moda Por eso ella no se enamora Estar soltera está de moda Por eso no va a cam-cam-cambiar Que nadie la reclame Celu en modo avión, no quiere que la llamen Le da lo mismo que la quieran o la amen Vacilar y joder, eso  son su plane  Y por ahora eso no va a cambiar Cero compromiso, sólo quiere bellaquear Porque no quiere que nadie le vuelva a fallar Beber y joder, no se va a amarrar Y por ahora eso no va a cambiar Cero compromiso, sólo quiere bellaquear Porque no quiere que nadie le vuelva a fallar Beber y joder, no se va a amarrar Ella lo que quiere e  joder, vacilar Solita pa  romper la disco Ella lo que quiere e  joder, vacilar (Chi-chi-chium) Dale hasta abajo, pa  hasta abajo sin parar Y estar soltera está de moda Por eso ella no se enamora Y estar soltera está de moda Por eso no va a cambiar (No va a cambiar) Estar soltera está de moda Por eso ella no se enamora (Rrá, rrá) Estar soltera está de moda (Chim) Por eso no va a cam-cam-cambiar Ponte-ponte pa  la vuelta, que yo voy pa l party Si no nos vemo , mami, en el after party Ponte pa l problema, ándañe, déjate ver Lo que aquí empezamo  lo matamo  en el motel Tú estás suelta por ahí, yo estoy suelto por acá Hacerte wiki wiki, como dice Yaviah Soltó el corazón, sacó a pasear la maldá  Tiene la mente dañá  A ella no hay quien la amanse Fuma y se va en un trance Perreando no pierde el balance To as le tiran y no están al alcance En el romance yo no creo que ella avance, yeah Y por ahora eso no va a cambiar Cero compromiso, sólo quiere bellaquear Porque no quiere que nadie le vuelva a fallar Beber y joder, no se va a amarrar Ella lo que quiere e  joder, vacilar Solita pa  romper la disco Ella lo que quiere e  joder, vacilar (Chi-chi-chium) Dale hasta abajo, pa  hasta abajo sin parar Y estar soltera está de moda Por eso ella no se enamora Y estar soltera está de moda Por eso no va a cambiar (No va a cambiar) Estar soltera está de moda Por eso ella no se enamora (Rrá, rrá) Estar soltera está de moda (Chim) Por eso no va a cam-cam-cambiar Ponte-ponte pa  la vuelta Ponte-ponte pa l party-pa l party, pa-party, el party Ponte-pon, Ponte pa l problema, ándale, déjate ver Ándale de-Ándale de-Ándale, déjate ver Yeah, yeah Lu-lu-lunay Chri-chri, Chri-Chri-Chris Jeday Yeah Gaby-gaby Music Dímelo Nino Magic Rhythms Yeah Lu-lu Pr en la casa Ey, candy, candy, dulce como candy Lu-lu-lunay "

In [122]:
pos = 10639
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10639,soltera,"lunay, daddy yankee",2019,2010,"Lu-lu-lunay Ella lo que quiere e joder, vacil..."


In [125]:
top10_df[top10_df['Lyrics'].str.contains('Somos 1 ft. One Man Army13/08Grupo Río')]

,Song,Performer,Year,Decade,Lyrics
10713,dame tu cosita,pitbull x el chombo x karol g,2018,2010,* = letra incompleta.01/08El Cadiz - Te Recuer...


In [126]:
lyrics = "Hah, bienvenidos a la cripta Mister Worldwide Todas las mujeres con las manos p arriba Todas las mujeres con las manos p arriba Boricua, colombiana, cubana, dominicana, venezolana Panameña, mexicana, blanquita, morena, canela, candela Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay! Dame tu cosita, ah, ah Dame tu cosita, ah, baby, give it to me Whippin  this on casa campo, yo soy el diablo, Romeo el santo Mister 305, Mister Worldwide All my business partners they call me Armando (Armando) Vamos pa  Caleta, es la grasa, bailando, gozando, acabando Beauty schools around the world Then if you got a question, ask Fernando Dale mamita, tú  tá rica, dame tu cosita, mmm Dale loquita, tú  tá rica, dame tu cosita, mmm Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay! Dame tu cosita, ah, ah Dame tu cosita, ah, give it to me Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay! Dame tu cosita, ah, ah (Múevete para aquí, muévete para allá) Cosita ¡Ay! Cosita Me mira y me dice:  qué rico, qué linda , que me mueva Y que la baile solo a él en la pista Que el Dj le de play a los cuentos de la penca Que ahora el Chombo dicen:  dame tu cosita  Llegaron los latinos, los griegos, los chinos Caribeños, africanos y de Estados Unidos To  el mundo anda pega o bailando este remix Karol G obviamente se pegó (¡ay!) Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay! Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay!, give it to me Cosita ¡Ay! Cosita If you get money, put your hands to the sky A ella le gusta dinero, dinero Dale más dinero A ella le gusta dinero, dinero Dale más dinero Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay! Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay!, give it to me Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay! Dame tu cosita, ah, ah (Muévete para aquí, muévete para allá) Dame tu cosita Ah, cosita (¡ay!) Dame tu cosita Ah, cosita (give it to me) Dame tu cosita, ah, ah Dame tu cosita, ah, ¡ay! Dame tu cosita, ah, ah Dame tu cosita"

In [127]:
pos = 10713
top10_df.loc[pos:pos,['Lyrics']] = [lyrics]
top10_df.loc[pos:pos]

,Song,Performer,Year,Decade,Lyrics
10713,dame tu cosita,pitbull x el chombo x karol g,2018,2010,"Hah, bienvenidos a la cripta Mister Worldwide ..."


In [128]:
dec = 2010
filename = "%s_lyrics.txt" % dec
file = open(FOLDER+filename, 'w')
_df = top10_df.query("Decade == @dec")
for l in _df.Lyrics:
    file.write(l+'\n')

## --------------------------------------------------------------------------------------

In [130]:
top10_df.query('Lyrics=="Error"')

,Song,Performer,Year,Decade,Lyrics


In [131]:
print("Total number of errors: ", len(top10_df.query('Lyrics=="Error"')))

Total number of errors:  0


We save the final dataframe to a csv file.

In [136]:
filename = "lyrics_per_song.csv"
out_file = open(FOLDER+filename, "w")

top10_df.to_csv(out_file, index=False)

## ------------------------------------------------------------------------------

In [39]:
#instrumentals

In [91]:
top10_df = top10_df.drop(top10_df.loc[13985:13985].index)

In [92]:
top10_df = top10_df.drop(top10_df.loc[36994:36994].index)

In [93]:
top10_df = top10_df.drop(top10_df.loc[36771:36771].index)

In [94]:
top10_df = top10_df.drop(top10_df.loc[36561:36561].index)

In [95]:
top10_df = top10_df.drop(top10_df.loc[20802:20802].index)